In [1]:
# =========================
# PHASE 0 — Prétraitement + DataLoaders + Modèle
# =========================
import os, shutil, random
import numpy as np
import torch
from zipfile import ZipFile
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from google.colab import drive

# 0.1 — Google Drive & unzip
drive.mount('/content/drive')
zip_path = "/content/drive/MyDrive/Colab Notebooks/COVID-19_Radiography_dataset.zip"
extract_path = "/content/COVID-19_Radiography_Dataset"
if not os.path.exists(extract_path):
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/")

# 0.2 — Réorganisation en 2 classes (COVID, Normal)
prepared_data_dir = "/content/covid_data_prepared"
os.makedirs(prepared_data_dir, exist_ok=True)
classes = ["COVID", "Normal"]

def find_src_dir(cls):
    candidates = [
        os.path.join("/content/COVID-19_Radiography_Dataset", cls, "images"),
        os.path.join("/content/COVID-19_Radiography_Dataset", cls),
        os.path.join("/content", cls, "images"),
        os.path.join("/content", cls),
    ]
    for p in candidates:
        if os.path.isdir(p):
            return p
    raise FileNotFoundError(f"Dossier images introuvable pour la classe '{cls}'.")

copied_counts = {}
for cls in classes:
    src_img_dir = find_src_dir(cls)
    dst_class_dir = os.path.join(prepared_data_dir, cls.replace(" ", "_"))
    os.makedirs(dst_class_dir, exist_ok=True)
    exts = {".png", ".jpg", ".jpeg", ".bmp"}
    n = 0
    for filename in os.listdir(src_img_dir):
        if os.path.splitext(filename.lower())[1] in exts:
            shutil.copy(os.path.join(src_img_dir, filename),
                        os.path.join(dst_class_dir, filename))
            n += 1
    copied_counts[cls] = n

print("✅ Réorganisation terminée. Structure ImageFolder prête.")
print("Comptes copiés:", copied_counts)

# 0.3 — Config / seeds / device
SEED = 42
IMG_SIZE = 224
BATCH_TRAIN = 32
BATCH_EVAL  = 128
WORKERS = 2 

def set_seeds(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  

set_seeds()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# 0.4 — Équilibrage + splits (stratifié)
base = datasets.ImageFolder(prepared_data_dir)
name_to_idx = base.class_to_idx
KEEP_NAMES = ["COVID", "Normal"]
KEEP_IDX = [name_to_idx[n] for n in KEEP_NAMES]

PER_CLASS = 500
VAL_FRAC  = 0.10
TEST_FRAC = 0.20
rng = np.random.RandomState(SEED)

idxs_by_class = {ci: [] for ci in KEEP_IDX}
for i, (_, y) in enumerate(base.samples):
    if y in KEEP_IDX:
        idxs_by_class[y].append(i)

kept_by_class = {}
for c, idxs in idxs_by_class.items():
    idxs = np.array(idxs); rng.shuffle(idxs)
    k = min(PER_CLASS, len(idxs))
    kept_by_class[c] = idxs[:k]

print("Après réduction :", {base.classes[c]: len(kept_by_class[c]) for c in KEEP_IDX},
      "Total:", sum(len(v) for v in kept_by_class.values()))

train_idx, val_idx, test_idx = [], [], []
for c in KEEP_IDX:
    idxs = kept_by_class[c].copy(); rng.shuffle(idxs)
    n = len(idxs)
    n_test = int(round(n * TEST_FRAC))
    n_val  = int(round((n - n_test) * VAL_FRAC))
    test_idx.extend(idxs[:n_test].tolist())
    val_idx.extend(idxs[n_test:n_test+n_val].tolist())
    train_idx.extend(idxs[n_test+n_val:].tolist())

print(f"Splits -> train:{len(train_idx)} | val:{len(val_idx)} | test:{len(test_idx)}")

# 0.5 — Transforms (train/val-test normalisés ; attack SANS Normalize)
MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])
eval_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])
attack_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

base_train  = datasets.ImageFolder(prepared_data_dir, transform=train_tfms)
base_eval   = datasets.ImageFolder(prepared_data_dir, transform=eval_tfms)
base_attack = datasets.ImageFolder(prepared_data_dir, transform=attack_tfms)

train_ds       = Subset(base_train,  train_idx)
val_ds         = Subset(base_eval,   val_idx)
test_ds        = Subset(base_eval,   test_idx)
attack_test_ds = Subset(base_attack, test_idx)

loader_kwargs = dict(pin_memory=(DEVICE.type=="cuda"))
if WORKERS > 0:
    loader_kwargs.update(num_workers=WORKERS, persistent_workers=True, prefetch_factor=2)

train_loader       = DataLoader(train_ds,       batch_size=BATCH_TRAIN, shuffle=True,  **loader_kwargs)
val_loader         = DataLoader(val_ds,         batch_size=BATCH_EVAL,  shuffle=False, **loader_kwargs)
test_loader        = DataLoader(test_ds,        batch_size=BATCH_EVAL,  shuffle=False, **loader_kwargs)
attack_test_loader = DataLoader(attack_test_ds, batch_size=BATCH_EVAL,  shuffle=False, **loader_kwargs)

print("✅ DataLoaders prêts.")

# 0.6 — Modèle
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2, dropout=0.3):
        super().__init__()
        self.b1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(inplace=True), nn.MaxPool2d(2)
        )
        self.b2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True), nn.MaxPool2d(2)
        )
        self.b3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1, bias=False),
            nn.BatchNorm2d(128), nn.ReLU(inplace=True), nn.MaxPool2d(2)
        )
        self.gap  = nn.AdaptiveAvgPool2d((1,1))
        self.drop = nn.Dropout(dropout)
        self.fc   = nn.Linear(128, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight); nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.b1(x); x = self.b2(x); x = self.b3(x)
        x = self.gap(x); x = torch.flatten(x, 1)
        x = self.drop(x); x = self.fc(x)
        return x

    def extract_features(self, x):
        x = self.b1(x); x = self.b2(x); x = self.b3(x)
        x = self.gap(x)
        return torch.flatten(x, 1)

model = SimpleCNN(num_classes=2, dropout=0.3).to(DEVICE)
model = model.to(memory_format=torch.channels_last)
print("✅ Modèle prêt.")


Mounted at /content/drive
✅ Réorganisation terminée. Structure ImageFolder prête.
Comptes copiés: {'COVID': 3616, 'Normal': 10192}
Device: cuda
Après réduction : {'COVID': 500, 'Normal': 500} Total: 1000
Splits -> train:720 | val:80 | test:200
✅ DataLoaders prêts.
✅ Modèle prêt.


In [2]:
# =========================
# PHASE 1 — Mixed Adversarial Training (AT)
# =========================
import time, copy
import torch
import torch.nn as nn
from torch.cuda.amp import GradScaler, autocast

# 1.1 — Optim / critères / early stop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
EPOCHS, PATIENCE = 25, 5
best_val, no_impr = float("inf"), 0
best_state = copy.deepcopy(model.state_dict())
scaler = GradScaler(enabled=(DEVICE.type=="cuda"))

# 1.2 — Espace NORMALISÉ (cohérent avec Normalize(MEAN,STD))
MEAN_T = torch.tensor(MEAN, device=DEVICE).view(1,3,1,1)
STD_T  = torch.tensor(STD,  device=DEVICE).view(1,3,1,1)
X_MIN  = (0 - MEAN_T) / STD_T
X_MAX  = (1 - MEAN_T) / STD_T

EPS_TRAIN   = 4/255
LAMBDA_ADV  = 0.3        
rng = np.random.RandomState(SEED)

@torch.no_grad()
def clamp_normed(x):
    return torch.max(torch.min(x, X_MAX), X_MIN)

def fgsm_norm(x, y, eps=EPS_TRAIN):
    model.eval()
    x_adv = x.detach().clone().requires_grad_(True)
    logits = model(x_adv)
    loss = criterion(logits, y)
    model.zero_grad(set_to_none=True)
    loss.backward()
    step = (eps/STD_T).to(x.device)
    x_adv = x_adv + step * x_adv.grad.detach().sign()
    x_adv = clamp_normed(x_adv).detach()
    model.train()
    return x_adv

def pgd_norm(x, y, eps=EPS_TRAIN, alpha=None, iters=3, random_start=True):
    if alpha is None: alpha = eps/4
    model.eval()
    x0 = x.detach()
    if random_start:
        delta0 = torch.empty_like(x0).uniform_(-eps, eps) / STD_T
        x_adv = clamp_normed(x0 + delta0)
    else:
        x_adv = x0.clone()
    for _ in range(iters):
        x_adv.requires_grad_(True)
        logits = model(x_adv)
        loss = criterion(logits, y)
        model.zero_grad(set_to_none=True)
        loss.backward()
        step = (alpha/STD_T).to(x.device)
        x_adv = x_adv + step * x_adv.grad.detach().sign()
        delta = torch.max(torch.min(x_adv - x0,  (eps/STD_T)), -(eps/STD_T))
        x_adv = clamp_normed(x0 + delta).detach()
    model.train()
    return x_adv

def bim_norm(x, y, eps=EPS_TRAIN, alpha=None, iters=5):
    if alpha is None: alpha = eps/10
    model.eval()
    x0 = x.detach(); x_adv = x0.clone()
    for _ in range(iters):
        x_adv.requires_grad_(True)
        logits = model(x_adv)
        loss = criterion(logits, y)
        model.zero_grad(set_to_none=True)
        loss.backward()
        step = (alpha/STD_T).to(x.device)
        x_adv = x_adv + step * x_adv.grad.detach().sign()
        delta = torch.max(torch.min(x_adv - x0,  (eps/STD_T)), -(eps/STD_T))
        x_adv = clamp_normed(x0 + delta).detach()
    model.train()
    return x_adv

def evaluate(model, loader):
    model.eval(); loss_sum=0.0; n=0; correct=0
    with torch.no_grad():
        for x,y in loader:
            x = x.to(DEVICE, memory_format=torch.channels_last, non_blocking=True)
            y = y.to(DEVICE)
            with autocast(enabled=(DEVICE.type=="cuda")):
                logits = model(x); loss = criterion(logits, y)
            loss_sum += loss.item()*y.size(0); n+=y.size(0)
            correct += (logits.argmax(1)==y).sum().item()
    return loss_sum/max(1,n), correct/max(1,n)

# 1.3 — Entraînement MAT (mélange clean + FGSM/PGD/BIM @ eps=4/255)
for epoch in range(1, EPOCHS+1):
    model.train(); t0=time.time(); run_loss=0.0; n=0
    for x,y in train_loader:
        x = x.to(DEVICE, memory_format=torch.channels_last, non_blocking=True)
        y = y.to(DEVICE)

        # attaque aléatoire par batch
        a = rng.choice(["fgsm","pgd","bim"])
        if a == "fgsm":
            x_adv = fgsm_norm(x, y, eps=EPS_TRAIN)
        elif a == "pgd":
            x_adv = pgd_norm(x, y, eps=EPS_TRAIN, alpha=EPS_TRAIN/4, iters=3, random_start=True)
        else:
            x_adv = bim_norm(x, y, eps=EPS_TRAIN,  alpha=EPS_TRAIN/10, iters=5)

        optimizer.zero_grad(set_to_none=True)
        with autocast(enabled=(DEVICE.type=="cuda")):
            logits_clean = model(x)
            logits_adv   = model(x_adv)
            loss_clean = criterion(logits_clean, y)
            loss_adv   = criterion(logits_adv,   y)
            loss = (1.0 - LAMBDA_ADV)*loss_clean + LAMBDA_ADV*loss_adv

        scaler.scale(loss).backward()
        scaler.step(optimizer); scaler.update()

        run_loss += loss.item()*y.size(0); n += y.size(0)

    train_loss = run_loss/max(1,n)
    val_loss, val_acc = evaluate(model, val_loader)
    print(f"[MAT] Epoch {epoch:02d} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | val_acc={val_acc:.3f} | {time.time()-t0:.1f}s")

    if val_loss < best_val - 1e-4:
        best_val = val_loss; no_impr = 0
        best_state = copy.deepcopy(model.state_dict())
    else:
        no_impr += 1
        if no_impr >= PATIENCE:
            print("Early stopping."); break

# 1.4 — Sauvegarde de l’état AT (pour Phase 2)
best_state_at = copy.deepcopy(best_state)
print("✅ AT terminé. Meilleur état sauvegardé dans best_state_at.")


/tmp/ipython-input-859165143.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(DEVICE.type=="cuda"))
/tmp/ipython-input-859165143.py:112: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(DEVICE.type=="cuda")):
/tmp/ipython-input-859165143.py:89: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(DEVICE.type=="cuda")):


[MAT] Epoch 01 | train_loss=0.7363 | val_loss=0.7099 | val_acc=0.650 | 9.8s
[MAT] Epoch 02 | train_loss=0.6735 | val_loss=0.6640 | val_acc=0.650 | 8.3s
[MAT] Epoch 03 | train_loss=0.6531 | val_loss=0.6547 | val_acc=0.650 | 7.5s
[MAT] Epoch 04 | train_loss=0.6674 | val_loss=0.6356 | val_acc=0.675 | 7.2s
[MAT] Epoch 05 | train_loss=0.6186 | val_loss=0.6729 | val_acc=0.688 | 7.7s
[MAT] Epoch 06 | train_loss=0.6310 | val_loss=0.6154 | val_acc=0.662 | 8.5s
[MAT] Epoch 07 | train_loss=0.6320 | val_loss=0.6415 | val_acc=0.713 | 8.3s
[MAT] Epoch 08 | train_loss=0.6266 | val_loss=0.6118 | val_acc=0.700 | 7.3s
[MAT] Epoch 09 | train_loss=0.6074 | val_loss=0.5989 | val_acc=0.688 | 7.6s
[MAT] Epoch 10 | train_loss=0.6266 | val_loss=0.6231 | val_acc=0.688 | 7.4s
[MAT] Epoch 11 | train_loss=0.6081 | val_loss=0.6213 | val_acc=0.700 | 9.1s
[MAT] Epoch 12 | train_loss=0.6086 | val_loss=0.5932 | val_acc=0.662 | 7.4s
[MAT] Epoch 13 | train_loss=0.6001 | val_loss=0.8594 | val_acc=0.625 | 8.3s
[MAT] Epoch 

In [3]:
# =========================
# PHASE 2 — Entraînement Student (Defensive Distillation)
# =========================
import copy, time, torch
import torch.nn as nn
from torch.cuda.amp import GradScaler, autocast

# --- 2.1 Teacher = modèle AT (gelé pour distillation)
teacher = SimpleCNN(num_classes=2, dropout=0.3).to(DEVICE).to(memory_format=torch.channels_last)
teacher.load_state_dict(best_state_at)
teacher.eval()

criterion_ce = nn.CrossEntropyLoss()

# --- Distillation loss
def make_distill_loss_fn(T=8.0, alpha=0.7):
    kldiv = nn.KLDivLoss(reduction="batchmean")
    def kd(student_logits, teacher_logits, y_true):
        with torch.no_grad():
            p_teacher = torch.softmax(teacher_logits / T, dim=1)
        log_p_student = torch.log_softmax(student_logits / T, dim=1)
        loss_distill = (T*T) * kldiv(log_p_student, p_teacher)
        loss_hard    = criterion_ce(student_logits, y_true)
        return alpha*loss_distill + (1.0-alpha)*loss_hard
    return kd

# --- éval rapide (loss/acc)
def evaluate_loss_acc(model, loader):
    model.eval(); loss_sum=0.0; n=0; correct=0
    with torch.no_grad():
        for x,y in loader:
            x = x.to(DEVICE, memory_format=torch.channels_last, non_blocking=True)
            y = y.to(DEVICE)
            logits = model(x)
            loss = criterion_ce(logits, y)
            loss_sum += loss.item()*y.size(0); n+=y.size(0)
            correct += (logits.argmax(1)==y).sum().item()
    return loss_sum/max(1,n), correct/max(1,n)

# --- entraînement du student
def train_student_distill(T=8.0, alpha=0.7, lr=1e-3, epochs=25, patience=5, tag="T8"):
    student = SimpleCNN(num_classes=2, dropout=0.3).to(DEVICE).to(memory_format=torch.channels_last)
    opt = torch.optim.AdamW(student.parameters(), lr=lr, weight_decay=1e-4)
    scaler = GradScaler(enabled=(DEVICE.type=="cuda"))
    kd_loss = make_distill_loss_fn(T=T, alpha=alpha)
    best_val, no_impr = float("inf"), 0
    best_state_s = copy.deepcopy(student.state_dict())

    for ep in range(1, epochs+1):
        student.train(); run_loss=0.0; n=0; t0=time.time()
        for x,y in train_loader:
            x = x.to(DEVICE, memory_format=torch.channels_last, non_blocking=True)
            y = y.to(DEVICE)
            with torch.no_grad():
                teacher_logits = teacher(x)
            opt.zero_grad(set_to_none=True)
            with autocast(enabled=(DEVICE.type=="cuda")):
                student_logits = student(x)
                loss = kd_loss(student_logits, teacher_logits, y)
            scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
            run_loss += loss.item()*y.size(0); n += y.size(0)

        train_loss = run_loss/max(1,n)
        val_loss, val_acc = evaluate_loss_acc(student, val_loader)
        print(f"[Student KD][{tag}] ep={ep:02d} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | val_acc={val_acc:.3f} | {time.time()-t0:.1f}s")

        if val_loss < best_val - 1e-4:
            best_val = val_loss; no_impr = 0
            best_state_s = copy.deepcopy(student.state_dict())
        else:
            no_impr += 1
            if no_impr >= patience:
                print("Early stopping."); break

    student.load_state_dict(best_state_s); student.eval()

    # Sauvegarder le student pour les cells d’évaluation indépendantes
    ckpt_path = f"/content/student_{tag}.pth"
    torch.save(student.state_dict(), ckpt_path)
    print(f"✅ Student sauvegardé: {ckpt_path}")
    return ckpt_path


ckpt_T8  = train_student_distill(T=8.0,  alpha=0.7, tag="T8")



/tmp/ipython-input-1979095028.py:44: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(DEVICE.type=="cuda"))
/tmp/ipython-input-1979095028.py:57: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(DEVICE.type=="cuda")):


[Student KD][T8] ep=01 | train_loss=0.3014 | val_loss=0.6510 | val_acc=0.650 | 6.8s
[Student KD][T8] ep=02 | train_loss=0.2472 | val_loss=0.6456 | val_acc=0.637 | 4.4s
[Student KD][T8] ep=03 | train_loss=0.2405 | val_loss=0.6820 | val_acc=0.675 | 3.0s
[Student KD][T8] ep=04 | train_loss=0.2336 | val_loss=0.6614 | val_acc=0.662 | 3.1s
[Student KD][T8] ep=05 | train_loss=0.2126 | val_loss=0.5948 | val_acc=0.700 | 4.4s
[Student KD][T8] ep=06 | train_loss=0.2165 | val_loss=0.6101 | val_acc=0.650 | 3.1s
[Student KD][T8] ep=07 | train_loss=0.2207 | val_loss=0.6320 | val_acc=0.650 | 3.0s
[Student KD][T8] ep=08 | train_loss=0.2084 | val_loss=0.6004 | val_acc=0.650 | 3.0s
[Student KD][T8] ep=09 | train_loss=0.2035 | val_loss=0.6125 | val_acc=0.725 | 4.5s
[Student KD][T8] ep=10 | train_loss=0.2047 | val_loss=0.6181 | val_acc=0.662 | 3.1s
Early stopping.
✅ Student sauvegardé: /content/student_T8.pth


In [4]:
!pip -q install adversarial-robustness-toolbox==1.17.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.8 MB/s eta 0:00:00


In [5]:
# =========================
# PHASE 2 — Évaluation FGSM / PGD / BIM (rapide)
# =========================
import numpy as np, torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, BasicIterativeMethod

# --- Recharger le student (choisir le checkpoint voulu)
ckpt_path = "/content/student_T8.pth" 
student = SimpleCNN(num_classes=2, dropout=0.3).to(DEVICE).to(memory_format=torch.channels_last)
student.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
student.eval()
print(f"✅ Student chargé depuis: {ckpt_path}")

criterion_ce = nn.CrossEntropyLoss()

# --- Utilitaires
def softmax_np(z):
    z = z - z.max(axis=1, keepdims=True)
    ez = np.exp(z); return ez / np.clip(ez.sum(axis=1, keepdims=True), 1e-12, None)

def build_art_classifier(pytorch_model):
    MEAN_VEC = np.array(MEAN, dtype=np.float32)
    STD_VEC  = np.array(STD,  dtype=np.float32)
    dummy_opt = torch.optim.SGD(pytorch_model.parameters(), lr=0.0)
    return PyTorchClassifier(
        model=pytorch_model,
        loss=criterion_ce,
        optimizer=dummy_opt,
        input_shape=(3, IMG_SIZE, IMG_SIZE),
        nb_classes=2,
        clip_values=(0.0, 1.0),
        preprocessing=(MEAN_VEC, STD_VEC)
    )

def loader_to_numpy(loader):
    xs, ys = [], []
    for xb, yb in loader:
        xs.append(xb.numpy()) 
        ys.append(yb.numpy())
    return np.concatenate(xs,0).astype(np.float32), np.concatenate(ys,0).astype(np.int64)

def metrics_from_probs(y_true, p1):
    y_pred = (p1 >= 0.5).astype(int)
    return dict(
        acc  = accuracy_score(y_true, y_pred),
        prec = precision_score(y_true, y_pred, zero_division=0),
        rec  = recall_score(y_true, y_pred, zero_division=0),
        f1   = f1_score(y_true, y_pred, zero_division=0),
        auc  = roc_auc_score(y_true, p1),
    )

# --- Éval CLEAN (utile pour référence)
@torch.no_grad()
def eval_clean(model, loader):
    model.eval(); logits_all, y_all = [], []
    for x,y in loader:
        x = x.to(DEVICE, memory_format=torch.channels_last, non_blocking=True)
        logits_all.append(model(x).cpu().numpy()); y_all.append(y.numpy())
    logits = np.concatenate(logits_all,0); y = np.concatenate(y_all,0)
    probs = softmax_np(logits); p1 = probs[:,1]
    return metrics_from_probs(y, p1)

print("CLEAN:", eval_clean(student, test_loader))

# --- Attaques FGSM / PGD / BIM
art_clf = build_art_classifier(student)
X_np, y_np = loader_to_numpy(attack_test_loader)

eps_list = [2/255, 4/255, 8/255]
results = {}

# FGSM
for eps in eps_list:
    atk = FastGradientMethod(estimator=art_clf, eps=eps, batch_size=BATCH_EVAL)
    X_adv = atk.generate(X_np)
    preds = art_clf.predict(X_adv)   
    p1 = preds[:, 1]                 
    results[f"FGSM@{eps:.5f}"] = metrics_from_probs(y_np, p1)

# PGD
for eps in eps_list:
    atk = ProjectedGradientDescent(estimator=art_clf, eps=eps, eps_step=eps/4,
                                   max_iter=20, targeted=False, num_random_init=0,
                                   batch_size=BATCH_EVAL)
    X_adv = atk.generate(X_np)
    preds = art_clf.predict(X_adv)
    p1 = preds[:, 1]
    results[f"PGD@{eps:.5f}"] = metrics_from_probs(y_np, p1)

# BIM
for eps in eps_list:
    atk = BasicIterativeMethod(estimator=art_clf, eps=eps, eps_step=eps/10,
                               max_iter=12, batch_size=BATCH_EVAL)
    X_adv = atk.generate(X_np)
    preds = art_clf.predict(X_adv)
    p1 = preds[:, 1]
    results[f"BIM@{eps:.5f}"] = metrics_from_probs(y_np, p1)


# --- Affichage
for k,v in results.items():
    print(f"{k:>12s} | acc={v['acc']:.4f} prec={v['prec']:.4f} rec={v['rec']:.4f} f1={v['f1']:.4f} auc={v['auc']:.4f}")


✅ Student chargé depuis: /content/student_T8.pth
CLEAN: {'acc': 0.71, 'prec': 0.7837837837837838, 'rec': 0.58, 'f1': 0.6666666666666666, 'auc': np.float64(0.7593000000000001)}


PGD - Batches:   0%|          | 0/2 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/2 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/2 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/2 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/2 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/2 [00:00<?, ?it/s]

FGSM@0.00784 | acc=0.6350 prec=0.6134 rec=0.7300 f1=0.6667 auc=0.6983
FGSM@0.01569 | acc=0.5350 prec=0.5207 rec=0.8800 f1=0.6543 auc=0.6166
FGSM@0.03137 | acc=0.5050 prec=0.5025 rec=1.0000 f1=0.6689 auc=0.4785
 PGD@0.00784 | acc=0.5250 prec=0.5210 rec=0.6200 f1=0.5662 auc=0.5400
 PGD@0.01569 | acc=0.3600 prec=0.4014 rec=0.5700 f1=0.4711 auc=0.3599
 PGD@0.03137 | acc=0.3050 prec=0.3488 rec=0.4500 f1=0.3930 auc=0.3554
 BIM@0.00784 | acc=0.5550 prec=0.5478 rec=0.6300 f1=0.5860 auc=0.5765
 BIM@0.01569 | acc=0.3650 prec=0.4015 rec=0.5500 f1=0.4641 auc=0.3714
 BIM@0.03137 | acc=0.3150 prec=0.3588 rec=0.4700 f1=0.4069 auc=0.3536


In [6]:
# =========================
# PHASE 2 — Évaluation C&W (coûteux, séparé)
# =========================
import numpy as np, torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import CarliniL2Method

# --- Recharger le student (choisir le checkpoint voulu)
ckpt_path = "/content/student_T8.pth"  
student = SimpleCNN(num_classes=2, dropout=0.3).to(DEVICE).to(memory_format=torch.channels_last)
student.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
student.eval()
print(f"✅ Student chargé depuis: {ckpt_path}")

criterion_ce = nn.CrossEntropyLoss()

def softmax_np(z):
    z = z - z.max(axis=1, keepdims=True)
    ez = np.exp(z); return ez / np.clip(ez.sum(axis=1, keepdims=True), 1e-12, None)

def build_art_classifier(pytorch_model):
    MEAN_VEC = np.array(MEAN, dtype=np.float32)
    STD_VEC  = np.array(STD,  dtype=np.float32)
    dummy_opt = torch.optim.SGD(pytorch_model.parameters(), lr=0.0)
    return PyTorchClassifier(
        model=pytorch_model,
        loss=criterion_ce,
        optimizer=dummy_opt,
        input_shape=(3, IMG_SIZE, IMG_SIZE),
        nb_classes=2,
        clip_values=(0.0, 1.0),
        preprocessing=(MEAN_VEC, STD_VEC)
    )

def loader_to_numpy(loader):
    xs, ys = [], []
    for xb, yb in loader:
        xs.append(xb.numpy())
        ys.append(yb.numpy())
    return np.concatenate(xs,0).astype(np.float32), np.concatenate(ys,0).astype(np.int64)

def metrics_from_probs(y_true, p1):
    y_pred = (p1 >= 0.5).astype(int)
    return dict(
        acc  = accuracy_score(y_true, y_pred),
        prec = precision_score(y_true, y_pred, zero_division=0),
        rec  = recall_score(y_true, y_pred, zero_division=0),
        f1   = f1_score(y_true, y_pred, zero_division=0),
        auc  = roc_auc_score(y_true, p1),
    )

# --- Éval CLEAN (référence)
@torch.no_grad()
def eval_clean(model, loader):
    model.eval(); logits_all, y_all = [], []
    for x,y in loader:
        x = x.to(DEVICE, memory_format=torch.channels_last, non_blocking=True)
        logits_all.append(model(x).cpu().numpy()); y_all.append(y.numpy())
    logits = np.concatenate(logits_all,0); y = np.concatenate(y_all,0)
    probs = softmax_np(logits); p1 = probs[:,1]
    return metrics_from_probs(y, p1)

print("CLEAN:", eval_clean(student, test_loader))

# --- Attaque C&W (L2)
art_clf = build_art_classifier(student)
X_np, y_np = loader_to_numpy(attack_test_loader)

cw_list = [0.3, 0.5] 
results_cw = {}

for c0 in cw_list:
    atk = CarliniL2Method(
        classifier=art_clf,
        initial_const=c0,
        binary_search_steps=1,   
        max_iter=20,
        learning_rate=0.01,
        targeted=False,
        batch_size=BATCH_EVAL
    )
    X_adv = atk.generate(X_np)
    preds = art_clf.predict(X_adv)  
    p1 = preds[:, 1]
    results_cw[f"CW@{c0:.2f}"] = metrics_from_probs(y_np, p1)


# --- Affichage
for k,v in results_cw.items():
    print(f"{k:>8s} | acc={v['acc']:.4f} prec={v['prec']:.4f} rec={v['rec']:.4f} f1={v['f1']:.4f} auc={v['auc']:.4f}")


✅ Student chargé depuis: /content/student_T8.pth
CLEAN: {'acc': 0.71, 'prec': 0.7837837837837838, 'rec': 0.58, 'f1': 0.6666666666666666, 'auc': np.float64(0.7593000000000001)}


C&W L_2:   0%|          | 0/2 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/2 [00:00<?, ?it/s]

 CW@0.30 | acc=0.6950 prec=0.7468 rec=0.5900 f1=0.6592 auc=0.7477
 CW@0.50 | acc=0.6950 prec=0.7468 rec=0.5900 f1=0.6592 auc=0.7475


In [7]:
# =========================
# PHASE 3 — Cell 3.1 : Génération adversariale (train/test)
# =========================
import numpy as np, torch
from torch.utils.data import Subset, DataLoader
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, BasicIterativeMethod, CarliniL2Method

# 3.1.1 — Charger le student à utiliser pour les embeddings et la prédiction
ckpt_path = "/content/student_T8.pth"  
student = SimpleCNN(num_classes=2, dropout=0.3).to(DEVICE).to(memory_format=torch.channels_last)
student.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
student.eval()
print(f"✅ Student chargé pour DAE: {ckpt_path}")

# 3.1.2 — Attack loaders (sans Normalize) pour TRAIN et TEST du détecteur
attack_train_ds = Subset(base_attack, train_idx)            
attack_train_loader = DataLoader(attack_train_ds, batch_size=BATCH_EVAL, shuffle=False, **loader_kwargs)


def loader_to_numpy(loader):
    xs, ys = [], []
    for x, y in loader:
        xs.append(x.numpy()) 
        ys.append(y.numpy())
    X = np.concatenate(xs, axis=0).astype(np.float32)
    y = np.concatenate(ys, axis=0).astype(np.int64)
    return X, y

X_clean_tr, y_clean_tr = loader_to_numpy(attack_train_loader)
X_clean_te, y_clean_te = loader_to_numpy(attack_test_loader)
print("Shapes clean:", X_clean_tr.shape, X_clean_te.shape)

# 3.1.3 — ART classifier (prend en charge la Normalize via preprocessing)
import torch.nn as nn
criterion_ce = nn.CrossEntropyLoss()
def build_art_classifier(pytorch_model):
    MEAN_VEC = np.array(MEAN, dtype=np.float32)
    STD_VEC  = np.array(STD,  dtype=np.float32)
    dummy_opt = torch.optim.SGD(pytorch_model.parameters(), lr=0.0)
    return PyTorchClassifier(
        model=pytorch_model,
        loss=criterion_ce,
        optimizer=dummy_opt,
        input_shape=(3, IMG_SIZE, IMG_SIZE),
        nb_classes=2,
        clip_values=(0.0, 1.0),
        preprocessing=(MEAN_VEC, STD_VEC)
    )

art_clf = build_art_classifier(student)

# 3.1.4 — Grilles d’attaque (train = light, test = complète)
ATTACK_GRID_TRAIN = {
    "FGSM": {"eps_list": [4/255]},
    "PGD":  {"eps_list": [4/255], "steps": 10, "step_frac": 0.25},
    "BIM":  {"eps_list": [4/255], "steps": 7,  "step_frac": 0.10},
}
ATTACK_GRID_TEST = {
    "FGSM": {"eps_list": [4/255, 8/255]},
    "PGD":  {"eps_list": [4/255, 8/255], "steps": 40, "step_frac": 0.25},
    "BIM":  {"eps_list": [4/255, 8/255], "steps": 10, "step_frac": 0.10},
    "CW":   {"initial_const": [0.3, 0.5]},  
}

def generate_adv_set(art_clf, X_np, y_np, attack_name, **kwargs):
    if attack_name == "FGSM":
        outs, ys, tags = [], [], []
        for eps in kwargs["eps_list"]:
            atk = FastGradientMethod(estimator=art_clf, eps=eps, batch_size=BATCH_EVAL)
            adv = atk.generate(X_np)
            outs.append(adv); ys.append(y_np); tags += [f"FGSM@{eps:.5f}"] * len(y_np)
        return np.concatenate(outs,0), np.concatenate(ys,0), np.array(tags)
    if attack_name == "PGD":
        outs, ys, tags = [], [], []
        for eps in kwargs["eps_list"]:
            step = eps * kwargs.get("step_frac", 0.25)
            atk = ProjectedGradientDescent(estimator=art_clf, eps=eps, eps_step=step,
                                           max_iter=kwargs.get("steps",40), targeted=False,
                                           num_random_init=1, batch_size=BATCH_EVAL)
            adv = atk.generate(X_np)
            outs.append(adv); ys.append(y_np); tags += [f"PGD@{eps:.5f}"] * len(y_np)
        return np.concatenate(outs,0), np.concatenate(ys,0), np.array(tags)
    if attack_name == "BIM":
        outs, ys, tags = [], [], []
        for eps in kwargs["eps_list"]:
            step = eps * kwargs.get("step_frac", 0.10)
            atk = BasicIterativeMethod(estimator=art_clf, eps=eps, eps_step=step,
                                       max_iter=kwargs.get("steps",10), targeted=False,
                                       batch_size=BATCH_EVAL)
            adv = atk.generate(X_np)
            outs.append(adv); ys.append(y_np); tags += [f"BIM@{eps:.5f}"] * len(y_np)
        return np.concatenate(outs,0), np.concatenate(ys,0), np.array(tags)
    if attack_name == "CW":
        outs, ys, tags = [], [], []
        for c0 in kwargs["initial_const"]:
            atk = CarliniL2Method(classifier=art_clf, initial_const=c0, binary_search_steps=1,
                                  max_iter=20, learning_rate=0.01, targeted=False,
                                  batch_size=BATCH_EVAL)
            adv = atk.generate(X_np)
            outs.append(adv); ys.append(y_np); tags += [f"CW@{c0:.2f}"] * len(y_np)
        return np.concatenate(outs,0), np.concatenate(ys,0), np.array(tags)
    raise ValueError("Attack inconnue:", attack_name)

def build_mixed_adv(art_clf, X_np, y_np, grid):
    XX, yy, src = [], [], []
    for name, cfg in grid.items():
        Xa, ya, tags = generate_adv_set(art_clf, X_np, y_np, name, **cfg)
        XX.append(Xa); yy.append(ya); src.append(tags)
    return np.concatenate(XX,0), np.concatenate(yy,0), np.concatenate(src,0)



✅ Student chargé pour DAE: /content/student_T8.pth
Shapes clean: (720, 3, 224, 224) (200, 3, 224, 224)


In [8]:
print("⚡ Génération adversaires pour TRAIN (détecteur, grille light)…")
X_adv_tr, y_adv_tr, src_tr = build_mixed_adv(art_clf, X_clean_tr, y_clean_tr, ATTACK_GRID_TRAIN)


⚡ Génération adversaires pour TRAIN (détecteur, grille light)…


PGD - Batches:   0%|          | 0/6 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
print("⚡ Génération adversaires pour TEST (détecteur & pipeline, grille complète)…")
X_adv_te, y_adv_te, src_te = build_mixed_adv(art_clf, X_clean_te, y_clean_te, ATTACK_GRID_TEST)


⚡ Génération adversaires pour TEST (détecteur & pipeline, grille complète)…


PGD - Batches:   0%|          | 0/2 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/2 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/2 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/2 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/2 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
print("Shapes adv:", X_adv_tr.shape, X_adv_te.shape)

Shapes adv: (2160, 3, 224, 224) (1600, 3, 224, 224)


In [13]:
# =========================
# PHASE 3 — Cell 3.2 : Détecteur (embeddings CNN → MLP) + calibration seuil
# =========================
import numpy as np, torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

# 3.2.1 — Utilitaires : normalisation et extraction des embeddings
MEAN_VEC = np.array(MEAN, dtype=np.float32)
STD_VEC  = np.array(STD,  dtype=np.float32)

def normalize_np_for_model(X_np):
   
   
    return (X_np - MEAN_VEC[None, :, None, None]) / STD_VEC[None, :, None, None]

@torch.no_grad()
def extract_embeddings(model, X_np, bs=256):
    model.eval(); embs = []
    for i in range(0, len(X_np), bs):
        xb = torch.from_numpy(normalize_np_for_model(X_np[i:i+bs])).to(DEVICE)
        xb = xb.to(memory_format=torch.channels_last, non_blocking=True)
        eb = model.extract_features(xb).float().cpu().numpy()
        embs.append(eb)
    return np.concatenate(embs, axis=0).astype(np.float32)

# 3.2.2 — Construit X/y pour le détecteur (0 = propre, 1 = adv)
Xemb_clean_tr = extract_embeddings(student, X_clean_tr)
Xemb_adv_tr   = extract_embeddings(student, X_adv_tr)
Xemb_clean_te = extract_embeddings(student, X_clean_te)
Xemb_adv_te   = extract_embeddings(student, X_adv_te)


mu = Xemb_clean_tr.mean(axis=0, keepdims=True)
sigma = Xemb_clean_tr.std(axis=0, keepdims=True) + 1e-6
def zscore(X): return (X - mu) / sigma


Xdet_tr = np.vstack([zscore(Xemb_clean_tr), zscore(Xemb_adv_tr)])
ydet_tr = np.concatenate([np.zeros(len(Xemb_clean_tr), dtype=np.int64),
                          np.ones(len(Xemb_adv_tr),   dtype=np.int64)])
Xdet_te = np.vstack([zscore(Xemb_clean_te), zscore(Xemb_adv_te)])
ydet_te = np.concatenate([np.zeros(len(Xemb_clean_te), dtype=np.int64),
                          np.ones(len(Xemb_adv_te),   dtype=np.int64)])

print("Embeddings:", Xdet_tr.shape, Xdet_te.shape)

# 3.2.3 — Détecteur MLP binaire
class DetectorMLP(nn.Module):
    """
    MLP plus expressif: 2 couches cachées + BatchNorm + Dropout
    in_dim -> 256 -> 64 -> 1
    """
    def __init__(self, in_dim, h1=256, h2=64, p=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, h1),
            nn.BatchNorm1d(h1),
            nn.ReLU(inplace=True),
            nn.Dropout(p),

            nn.Linear(h1, h2),
            nn.BatchNorm1d(h2),
            nn.ReLU(inplace=True),
            nn.Dropout(p),

            nn.Linear(h2, 1)  
            
        )

    def forward(self, x):
        return self.net(x).squeeze(1)

det_in = Xdet_tr.shape[1]
detector = DetectorMLP(det_in, h1=256, h2=64, p=0.3).to(DEVICE)


det_opt  = torch.optim.AdamW(detector.parameters(), lr=2e-3, weight_decay=1e-4)
det_crit = nn.BCEWithLogitsLoss()


def to_loader_feats(X, y, bs=256, shuffle=False):
    ds = TensorDataset(torch.from_numpy(X).float(), torch.from_numpy(y).float())
    return DataLoader(ds, batch_size=bs, shuffle=shuffle, num_workers=0)

dl_tr = to_loader_feats(Xdet_tr, ydet_tr, bs=256, shuffle=True)
dl_te = to_loader_feats(Xdet_te, ydet_te, bs=512, shuffle=False)

def train_epoch(model, loader, opt, crit):
    model.train(); loss_sum=0.0; n=0
    for xb, yb in loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        opt.zero_grad()
        logits = model(xb)
        loss = crit(logits, yb)
        loss.backward(); opt.step()
        loss_sum += loss.item()*xb.size(0); n += xb.size(0)
    return loss_sum/max(1,n)

@torch.no_grad()
def infer_logits(model, X):
    model.eval(); outs=[]
    for i in range(0, len(X), 512):
        xb = torch.from_numpy(X[i:i+512]).float().to(DEVICE)
        outs.append(model(xb).cpu().numpy())
    return np.concatenate(outs)

# 3.2.4 — Entraînement + calibration seuil (max F1 sur set TEST du détecteur)
EPOCHS_DET = 20
best_f1, best_state = -1, None
for ep in range(1, EPOCHS_DET+1):
    tr_loss = train_epoch(detector, dl_tr, det_opt, det_crit)
    
    logits_te = infer_logits(detector, Xdet_te)
    prob_te = 1.0 / (1.0 + np.exp(-logits_te))
    yhat_05 = (prob_te > 0.5).astype(int)
    f1_now  = f1_score(ydet_te.astype(int), yhat_05.astype(int), zero_division=0)
    print(f"[DET] ep={ep:02d} | train_loss={tr_loss:.4f} | f1@0.5={f1_now:.3f}")
    if f1_now > best_f1:
        best_f1 = f1_now
        best_state = {k: v.detach().cpu().clone() for k,v in detector.state_dict().items()}


detector.load_state_dict({k: v.to(DEVICE) for k, v in best_state.items()})




logits_te = infer_logits(detector, Xdet_te)
prob_te = 1.0 / (1.0 + np.exp(-logits_te))
clean_mask = (ydet_te == 0)

target_fpr = 0.15   
thr = np.sort(prob_te)
best_tau, best_tpr = 0.5, -1
for t in thr:
    fpr_clean = (prob_te[clean_mask] > t).mean()
    if fpr_clean <= target_fpr:
        tpr_adv = (prob_te[~clean_mask] > t).mean()
        if tpr_adv > best_tpr:
            best_tpr, best_tau = tpr_adv, float(t)

print(f"✅ Seuil calibré : τ = {best_tau:.3f} (FPR ≤ {target_fpr*100:.0f}%, TPR adv ≈ {best_tpr*100:.1f}%)")


torch.save(detector.state_dict(), "/content/detector_mlp.pth")
with open("/content/detector_tau.txt", "w") as f:
    f.write(str(best_tau))
print("✅ Détecteur sauvegardé (/content/detector_mlp.pth) & τ écrit (/content/detector_tau.txt)")


Embeddings: (2880, 128) (1800, 128)
[DET] ep=01 | train_loss=0.5063 | f1@0.5=0.880
[DET] ep=02 | train_loss=0.2974 | f1@0.5=0.857
[DET] ep=03 | train_loss=0.2230 | f1@0.5=0.858
[DET] ep=04 | train_loss=0.1696 | f1@0.5=0.861
[DET] ep=05 | train_loss=0.1265 | f1@0.5=0.858
[DET] ep=06 | train_loss=0.0897 | f1@0.5=0.857
[DET] ep=07 | train_loss=0.0801 | f1@0.5=0.858
[DET] ep=08 | train_loss=0.0718 | f1@0.5=0.858
[DET] ep=09 | train_loss=0.0608 | f1@0.5=0.857
[DET] ep=10 | train_loss=0.0484 | f1@0.5=0.857
[DET] ep=11 | train_loss=0.0443 | f1@0.5=0.856
[DET] ep=12 | train_loss=0.0477 | f1@0.5=0.857
[DET] ep=13 | train_loss=0.0425 | f1@0.5=0.857
[DET] ep=14 | train_loss=0.0403 | f1@0.5=0.858
[DET] ep=15 | train_loss=0.0356 | f1@0.5=0.860
[DET] ep=16 | train_loss=0.0308 | f1@0.5=0.858
[DET] ep=17 | train_loss=0.0307 | f1@0.5=0.861
[DET] ep=18 | train_loss=0.0307 | f1@0.5=0.857
[DET] ep=19 | train_loss=0.0273 | f1@0.5=0.857
[DET] ep=20 | train_loss=0.0265 | f1@0.5=0.857
✅ Seuil calibré : τ = 0.

In [15]:
# =========================
# PHASE 3 — Cell 3.3 : Évaluation pipeline global
# =========================
import numpy as np, torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# 3.3.1 — Recharger détecteur & seuil (si besoin)
detector = DetectorMLP(det_in, h1=256, h2=64, p=0.3).to(DEVICE)
detector.load_state_dict(torch.load("/content/detector_mlp.pth", map_location=DEVICE))
detector.eval()
with open("/content/detector_tau.txt","r") as f:
    TAU = float(f.read().strip())
print(f"ℹ️  τ utilisé pour le pipeline = {TAU:.3f}")

@torch.no_grad()
def predict_classes(model, X_np, bs=256):
    model.eval(); preds, probs = [], []
    for i in range(0, len(X_np), bs):
        xb = torch.from_numpy(normalize_np_for_model(X_np[i:i+bs])).to(DEVICE)
        xb = xb.to(memory_format=torch.channels_last, non_blocking=True)
        logits = model(xb)
        pb = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
        yh = logits.argmax(1).cpu().numpy()
        probs.append(pb); preds.append(yh)
    return np.concatenate(preds), np.concatenate(probs)

# 3.3.2 — Construire le set TEST global (propres + adversaires)
X_test_global = np.vstack([X_clean_te, X_adv_te])    
y_is_adv      = np.concatenate([np.zeros(len(X_clean_te), dtype=np.int64),
                                np.ones(len(X_adv_te),  dtype=np.int64)])
y_true_cls    = np.concatenate([y_clean_te, y_adv_te])

# 3.3.3 — Détection (seuil τ)
Xemb_global = extract_embeddings(student, X_test_global)
Xemb_global_z = (Xemb_global - mu) / sigma

with torch.no_grad():
    det_logits = []
    for i in range(0, len(Xemb_global_z), 512):
        xb = torch.from_numpy(Xemb_global_z[i:i+512]).to(DEVICE).float()
        det_logits.append(detector(xb).cpu().numpy())
    det_logits = np.concatenate(det_logits)

det_prob = 1.0 / (1.0 + np.exp(-det_logits))
det_pred = (det_prob > TAU).astype(int) 

# 3.3.4 — Classification uniquement sur les ACCEPTÉES
accepted_mask = (det_pred == 0)
X_accepted    = X_test_global[accepted_mask]
y_true_acc    = y_true_cls[accepted_mask]
y_is_adv_acc  = y_is_adv[accepted_mask]
yhat_acc, _   = predict_classes(student, X_accepted)


n_adv_total       = int((y_is_adv == 1).sum())
n_adv_blocked     = int(((y_is_adv == 1) & (det_pred == 1)).sum())
pct_adv_blocked   = 100.0 * n_adv_blocked / max(1, n_adv_total)

clean_acc_mask    = (y_is_adv_acc == 0)
n_clean_accepted  = int(clean_acc_mask.sum())
n_clean_correct   = int((yhat_acc[clean_acc_mask] == y_true_acc[clean_acc_mask]).sum())
pct_clean_correct = 100.0 * n_clean_correct / max(1, n_clean_accepted)

n_clean_total     = int((y_is_adv == 0).sum())
n_clean_blocked   = int(((y_is_adv == 0) & (det_pred == 1)).sum())
pct_false_rejects = 100.0 * n_clean_blocked / max(1, n_clean_total)

print("\n🔎 Pipeline global (TEST étendu)")
print(f"• % d’adversariales bloquées               : {pct_adv_blocked:.2f}%  ({n_adv_blocked}/{n_adv_total})")
print(f"• % d’images propres correctement classées : {pct_clean_correct:.2f}%  ({n_clean_correct}/{max(1,n_clean_accepted)})")
print(f"• % de faux rejets (propres bloquées)      : {pct_false_rejects:.2f}%  ({n_clean_blocked}/{n_clean_total})")

# 3.3.5 — Breakdown par type d’attaque (sur la partie adversariale)
start_adv = len(X_clean_te)
det_pred_adv = det_pred[start_adv:]
for tag in np.unique(src_te):
    m = (src_te == tag)
    n_tot = int(m.sum())
    n_blk = int((det_pred_adv[m] == 1).sum())
    print(f"- {tag:>10s}: bloquées {n_blk}/{n_tot}  ({100.0*n_blk/max(1,n_tot):.1f}%)")


ℹ️  τ utilisé pour le pipeline = 0.537

🔎 Pipeline global (TEST étendu)
• % d’adversariales bloquées               : 77.62%  (1242/1600)
• % d’images propres correctement classées : 71.76%  (122/170)
• % de faux rejets (propres bloquées)      : 15.00%  (30/200)
- BIM@0.01569: bloquées 187/200  (93.5%)
- BIM@0.03137: bloquées 198/200  (99.0%)
-    CW@0.30: bloquées 30/200  (15.0%)
-    CW@0.50: bloquées 30/200  (15.0%)
- FGSM@0.01569: bloquées 199/200  (99.5%)
- FGSM@0.03137: bloquées 200/200  (100.0%)
- PGD@0.01569: bloquées 198/200  (99.0%)
- PGD@0.03137: bloquées 200/200  (100.0%)
